In [30]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import csv
import os
import glob


In [31]:
#validation data:

val_coords = {'B755':{'vert_lines':[1703, 3110], 'hor_lines':[1273]},
'B756': {'vert_lines':[1933], 'hor_lines':[1299]},
'B757': {'vert_lines':[910, 2293], 'hor_lines':[1266]},
'B761': {'vert_lines':[2286], 'hor_lines':[1246]},
'B762': {'vert_lines':[1270, 2576], 'hor_lines':[1266]},
'B763': {'vert_lines':[1706, 3026], 'hor_lines':[2203]},
'B766': {'vert_lines':[1463, 2886], 'hor_lines':[1256]},
'B773': {'vert_lines':[950, 2306], 'hor_lines':[1246]},
'B780': {'vert_lines':[2123], 'hor_lines':[1250]},
'B782': {'vert_lines':[1370, 2936], 'hor_lines':[1236]},
'B784': {'vert_lines':[2150], 'hor_lines':[1243]},
'B786': {'vert_lines':[1293, 2809], 'hor_lines':[1250]},
'B788': {'vert_lines':[2173], 'hor_lines':[1219]},
'B790': {'vert_lines':[1203, 2810], 'hor_lines':[1256]},
'B765': {'vert_lines':[2310], 'hor_lines':[1259]},
'B768': {'vert_lines':[2100], 'hor_lines':[1263]},
'B769': {'vert_lines':[2383], 'hor_lines':[1266]},
'B770': {'vert_lines':[2733, 1383], 'hor_lines':[1276]},
'B772': {'vert_lines':[2046], 'hor_lines':[1263]},
'B774': {'vert_lines':[1463, 2783], 'hor_lines':[1256]},
'B775': {'vert_lines':[1789], 'hor_lines':[1236]},
'B776': {'vert_lines':[1956], 'hor_lines':[1273]},
'B777': {'vert_lines':[2369], 'hor_lines':[1253]},
'B800': {'vert_lines':[1436, 2766], 'hor_lines':[1263]},
'B901': {'vert_lines':[1683], 'hor_lines':[1236]},
'B902': {'vert_lines':[2089], 'hor_lines':[1279]},    #64 lines above here

    }
val_names = val_coords.keys()


In [32]:
#image preprocessing functions
#if usig BGR masks, not needed if regular grayscale ones are used
def ConvertMasksBGR(BGR_img) :
    lower_pmts = np.array([250, 0, 0], dtype = "uint16")
    upper_pmts = np.array([255,0,0], dtype = "uint16")
    pmt_mask = cv2.inRange(BGR_img, lower_pmts, upper_pmts)
    pmt_mask = cv2.cvtColor(pmt_mask, cv2.COLOR_GRAY2BGR)
   # print(pmt_mask.shape)
    pmt_mask[pmt_mask == 255] = 1
    return pmt_mask

####################################################################################

def UndistortImage(orig) :
    #undistorted = orig.copy();

 # //Mat cameraMatrix = Mat::eye( 3, 3, CV_64F ); //K
 # //Mat distCoeffs = Mat::zeros( 1, 4, CV_64F ); //D
 # //  Here's the constants extracted directly from MATLAB:
 # //           FocalLength: [2.7605e+03 2.7670e+03]
 # //        PrincipalPoint: [1.9143e+03 1.5964e+03]
 # //             ImageSize: [3000 4000]
 # //      RadialDistortion: [-0.2398 0.1145]
 # //  TangentialDistortion: [0 0]
 # //               Skew: 0
 # //  
 # // The intrinsics fx, fy, cx and cy in your intrinsics matrix are the
 # // focal length and principle point above, and are dependant on the
  #// image size so best to make sure the images you're using haven't
  #// been rescaled.
  ##// The four values only give the minimum k1, k2, p1, p2 that you need 
  #// because calibrating additional parameters weren't enabled by the 
  #// students when they did this calibration. Tangential distortion is set to 
  #// zero because it was also disabled.  

    cameraMatrix = np.array([[ 2.7605e+03,      0,          1.9143e+03],
                        [0,               2.7670e+03,      1.5964e+03], 
                        [0,                    0,               1]] )
    
    distCoeffs = np.array( [-0.2398, 0.1145, 0, 0] )
 #   print(cameraMatrix)
    undistorted = cv2.undistort(orig, cameraMatrix, distCoeffs);

    return undistorted;


In [33]:
#peak picking funcs


#stolen from stack overflow 
#this is set up for a multi peak gaussian fit 
def peaks_func(x, *params):
    #print(params)
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        ctr = params[i]
        amp = params[i+1]
        wid = params[i+2]
        y = y + amp * np.exp( -((x - ctr)/wid)**2)
    return y


#################################################################
#guess locations of peaks using a threshold and data above, 
#instead of fitting to peaks, just see where peak goes above and below average and get centre of those two point

def peak_guess(x_dat, y_dat) : 
    tempx = []
    tempy = []
    peak_loc = []
    
    #if it is above this it is a peak
    #peak_thresh = np.min(y_dat) + (np.max(y_dat) - np.min(y_dat))*0.5
    peak_thresh = np.average(y_dat)
    for i in range(len(y_dat)) :
        if y_dat[i] >= peak_thresh :
            on_peak = True
            tempx.append(x_dat[i])
            tempy.append(y_dat[i])
        else :
        #should only do this AFTER end of peak
            if len(tempx) > 10 : #no empty data
                #print([tempy[0], tempy[-1], len(tempy)])

                #if int(np.abs(tempy[0] - tempy[-1])) < 500: #check that it is a full peak
                peak_start = tempx[0]
                peak_end = tempx[-1]
                peak_cen = peak_start + (peak_end - peak_start) / 2
                peak_max = np.max(tempy)
                peak_wid = peak_end - peak_start

                peak_loc.append([peak_cen, peak_max, peak_wid])
            tempx = []
            tempy = []
    return peak_loc, peak_thresh

####################################################################################


In [34]:
#drawing various lines 
#peaks is the pmt lines, margins are the image crops that remove the edges of images and need to be accounted for 
#super module lines are recalculated in here

def draw_lines(img_in, peaks1, peaks2, margin1, margin2,  draw_pmt_lines = True) :
    global smod_count_num
    global smod_count_lines
    
    #make a new instance, may be unnecessary but really dont want to draw over data 
    img = img_in.copy()
    if draw_pmt_lines == True :
        if len(peaks1) > 8 :
            num_rem1 = 2
        else :
            num_rem1 = 1
        
        if len(peaks2) > 8:
            num_rem2 = 2
        else :
            num_rem2 = 1
                       
            
       # cv2.line(img, (0, int(peaks1[0])), (img.shape[1], int(peaks1[0])),  color=[255, 0, 255] , thickness=2)
       # cv2.line(img, (0, int(peaks1[-1])), (img.shape[1], int(peaks1[-1])),  color=[255, 0, 255] , thickness=2)

        for i in range(0, len(peaks1)) :
            if (i < num_rem1) or (i >= len(peaks1)-num_rem1):
                cv2.line(img, (0, int(peaks1[i]) + margin1), (img.shape[1], int(peaks1[i] + margin1)),  color=[255, 0, 255] , thickness=2)
            else :
                cv2.line(img, (0, int(peaks1[i]) + margin1), (img.shape[1], int(peaks1[i] + margin1)),  color=[0, 0, 255] , thickness=2)

        for j in range(0, len(peaks2)) :
            if (j < num_rem2) or (j >= len(peaks2)-num_rem2) :
                cv2.line(img, (int(peaks2[j] + margin2), 0), (int(peaks2[j] + margin2), img.shape[0]),  color=[255, 0, 255] , thickness=2) #PUT IN ANGLE CORRECTION
            else :
                cv2.line(img, (int(peaks2[j] + margin2), 0), (int(peaks2[j] + margin2), img.shape[0]),  color=[0, 0, 255] , thickness=2)
                
                
    super_x, gaps = super_lines(peaks1, "x")
    super_y, gaps = super_lines(peaks2, "y")   
    
    for k in range(0, len(super_x)) :
        cv2.line(img, (0, int(super_x[k] + margin1)), (img.shape[1], int(super_x[k] + margin1)),  color=[0, 255, 0] , thickness=4)
        
    for l in range(0, len(super_y)) :
        
        cv2.line(img, (int(super_y[l] + margin2), 0), (int(super_y[l] + margin2), img.shape[0]),  color=[0, 255, 0] , thickness=4)
        #cv2.putText(img, f'{smod_count_num}', (int(super_y[l] + margin2), int(img.shape[0]/2 + margin2)), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 2)
        #smod_count_lines.append([super_y[l], smod_count_num])
        #smod_count_num += 1
            
    return img



In [44]:
######Questionable methods begin here
def super_lines(pmt_lines_long, axis) :
    
    #_long is all lines, remove outer lines as they can be wrong
    super_mod_lines = []
    if len(pmt_lines_long) > 8 :
        num_rem = 2
    else :
        num_rem = 1

    pmt_lines = pmt_lines_long[num_rem:-num_rem]

    #for i in range(0, len(popt), 3) :
     #   pmt_lines.append(popt[i])
    
    #IT WOULD BE NICE TO GET AN IMAGE INDEPENDEDNT MEASURE OF GAPS E.G. BASED OFF OF PMT WIDTH  
    gaps_long = np.diff(pmt_lines_long)
    gaps = np.diff(pmt_lines)
    
    #Pattern test horizontal lines
    #have three possibilities: 1st, 2nd, 3rd gap is a super module
    #split gap dat into predicted large and small gaps
    #small gap data is more reliable as there are more of them
    #True small gap data will have small difference between gap sizes, false ones will contain large and small gaps so difference will be larger
    #there is probably a case where this is not true
    if axis == "x" :
        #hypothesis that first gap is super module
        
        hyp1 = {'sindx':range((num_rem)%3,len(gaps_long), 3), 'hi':gaps[::3], 'lo':np.setdiff1d(gaps,gaps[::3]), 
                'long_hi':gaps_long[(num_rem)%3::3]}
        hyp2 = {'sindx':range((1 + num_rem)%3,len(gaps_long), 3), 'hi':gaps[1::3], 'lo':np.setdiff1d(gaps,gaps[1::3]), 
                'long_hi':gaps_long[(num_rem+1)%3::3]}
        hyp3 = {'sindx':range((2 + num_rem)%3,len(gaps_long), 3), 'hi':gaps[2::3], 'lo':np.setdiff1d(gaps,gaps[2::3]),
                'long_hi':gaps_long[(num_rem+2)%3::3]}
        
        hyps = [hyp1, hyp2, hyp3]
 #       print(hyps)
        scores =[] 
        for i in range(len(hyps)):
                scores.append(np.max(hyps[i]['lo']) - np.min(hyps[i]['lo']))
                #scores.append(np.max(hyps[i]['hi'])) 
        best = scores.index(min(scores))
        

    #Simpler: average of large gap data will be larger than average of small gap
    #more gaps avai;lable so I think this is safe
    if axis == "y" :
        #ase for every other gap (modules)
      #  hyp1 = {'sindx':range(0,len(gaps), 2),'hi':gaps[::2], 'lo':np.setdiff1d(gaps,gaps[::2])}
      #  hyp2 = {'sindx':range(1,len(gaps), 2),'hi':gaps[1::2], 'lo':np.setdiff1d(gaps,gaps[1::2])}
      #  hyps = [hyp1, hyp2]
      #  if np.average(hyp1['hi']) > np.average(hyp2['hi']) :
      #      best = 0
      #  else :
      #      best = 1
    
    
    
        #mark supermodules like previously
        hyp1 = {'sindx':range((num_rem)%4,len(gaps_long), 4),'hi':gaps[::4], 'lo':np.setdiff1d(gaps,gaps[::4]), 
                'long_hi':gaps_long[(num_rem)%4::4]}
        hyp2 = {'sindx':range((1 + num_rem)%4,len(gaps_long), 4),'hi':gaps[1::4], 'lo':np.setdiff1d(gaps,gaps[1::4]), 
                'long_hi':gaps_long[(num_rem+1)%4::4]}
        hyp3 = {'sindx':range((2 + num_rem)%4,len(gaps_long), 4),'hi':gaps[2::4], 'lo':np.setdiff1d(gaps,gaps[2::4]),
                'long_hi':gaps_long[(num_rem+2)%4::4]}
        hyp4 = {'sindx':range((3 + num_rem)%4,len(gaps_long), 4),'hi':gaps[3::4], 'lo':np.setdiff1d(gaps,gaps[3::4]),
                'long_hi':gaps_long[(num_rem+3)%4::4]}
        
        hyps = [hyp1, hyp2, hyp3, hyp4]

        #print(gaps)
        #print(hyp1['lo'])
        #print(hyp2['lo'])
        #print(hyp3['lo'])
        #print(hyp4['lo'])

        
        scores =[] 
        for j in range(len(hyps)):
                #scores.append(np.max(hyps[j]['lo']) - np.min(hyps[j]['lo']))
                #scores.append(np.max(hyps[j]['hi'])) 
                scores.append(np.average(hyps[j]['lo']))
            
        best = scores.index(min(scores))
      # print(f'y hypothesis:{hyps}')        
    
            
    
    for k in hyps[best]['sindx'] :
        if k != len(pmt_lines_long) :
            super_mod_lines.append(pmt_lines_long[k] + (pmt_lines_long[k+1] - pmt_lines_long[k])/2)
    
    return super_mod_lines, hyps[best]['hi']
#################################################################
        

In [45]:
def get_scores(img, angle) :
    #get grayscale so only 2d matrix
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    img_size = gray_img.shape
    x_score = []
    y_score = []
    mult = 2    #how many lines skipped in image scan (1 is dto get a score for every line)
    thresh = 1000    #threshold for pixel scores


    #loop over a and y with lines drawn on a mask to get score from pixel sums
   # print(img_size)
    for i in np.linspace(0, img_size[0], int(img_size[0]/mult)) :
        img_mask = np.zeros(img_size,dtype=np.uint8)
        start_y = int(i)
        end_y = start_y + int(angle*img_size[1])  #minus because coords measured from top left corner
        cv2.line(img_mask, (0, start_y), (img_size[1], end_y),  color=1 , thickness=1)


        fit = gray_img*img_mask
        
        #DEBUG
        #print(np.unique(fit))
        #if(fit.all()<1) :
        #    continue     
        #init_rad = circ_r
      # cv2.imshow("img", gray_img)
      # cv2.waitKey(0)
    
        score = np.sum(fit)
        x_score.append(score)
        

    for j in np.linspace(0, img_size[1], int(img_size[1]/mult)) :
        img_mask = np.zeros(img_size,dtype=np.uint8)
        start_x = int(j)
        end_x = start_x - int(angle*img_size[0])
        cv2.line(img_mask, (start_x, 0), (end_x, img_size[0]),  color=1 , thickness=1)

        fit = gray_img*img_mask

        score = np.sum(fit)
        y_score.append(score)

    return x_score, y_score

In [46]:
#UPDAATE this code to not use circles, maximise score, may be longer but faaar better
def get_angle(in_img) :
    
    img = in_img.copy()
    
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_size = gray_img.shape
    gray_img[gray_img > 0] = 100
    gray_img = gray_img[500:-500, :] 
   # print(np.unique(gray_img))
    edges = cv2.Canny(gray_img,0,100)
    circles = cv2.HoughCircles(edges,cv2.HOUGH_GRADIENT,dp=1,
                           minDist=200,
                           param1=100,
                           param2=20,
                           minRadius=140,
                           maxRadius=200
                          )
        #print(circles)
        
    #x, y data for circle centers
    circles = np.uint16(np.around(circles))
    circ_x = []
    circ_y = []
    for i in circles[0, :] :
        cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
        #print(i[0], i[1])
        circ_x.append(i[0])
        circ_y.append(i[1])
  #  print(np.min(circ_y))
  #  print(np.max(circ_y))
    
    
    #Get the first row of circles
    circ_x_first = []
    circ_y_first = []
    min_y = np.min(circ_y)
    
    #for drawing circles
    for i in range(len(circ_y)) :
        if circ_y[i] <  2*min_y :
         #   circ_y.pop(i)
          #  circ_x.pop(i)
            circ_x_first.append(circ_x[i])
            circ_y_first.append(circ_y[i])
                                
    for i in range(len(circ_x_first)) :
        cv2.circle(img, (circ_x_first[i], circ_y_first[i]), 100, (0,255,0), -1)
    
    #straight line function
    def func(x, a, b) :
        return a*x + b
    popt, pcov = curve_fit(func, circ_x_first, circ_y_first)
    
    
   #print(popt)
    #print(circ_x)
    #print(circ_y)
    #cv2.imshow('img', img)
    #cv2.waitKey(0)
    return popt[0]

In [47]:
def plot_data(x1, y1, pmt_lines_1, x2, y2, pmt_lines_2) :

    #IT WOULD BE NICE TO GET AN IMAGE INDEPENDEDNT MEASURE OF GAPS E.G. BASED OFF OF PMT WIDTH  
    gaps1 = np.diff(pmt_lines_1)
    gaps2 = np.diff(pmt_lines_2)
    gap_x_1 = []
    gap_x_2 = []
    
    for k in range(len(gaps1)) :
        gap_x_1.append(pmt_lines_1[k] + gaps1[k]/2)
        
    for l in range(len(gaps2)) :
        gap_x_2.append(pmt_lines_2[l] + gaps2[l]/2)
    
    #plot setup
    fig, axs = plt.subplots(2, 2, figsize=(10,6), sharex='col')
    fig.tight_layout()
    plt.subplots_adjust(left = 0.1, bottom = 0.1, hspace=0.4, wspace=0.2)


    axs[0, 0].plot(x1, y1) #line scores
    for i in pmt_lines_1 :
        axs[0, 0].axvline(x=i)
    #axs[0, 0].axhline( thresh1, color = 'g', label='thresh')
    axs[0,0].set_title("Horizontal line scan")
    axs[0,0].set_xlabel("X position (px)")
    axs[0,0].set_ylabel("Score (px)")

    
    axs[0, 1].plot(x2, y2) 
    for j in pmt_lines_2 :
        axs[0, 1].axvline(x=j)

    #axs[0, 1].axhline( thresh2, color = 'g', label='thresh')
    axs[0, 1].set_title("Vertical line scan")
    axs[0,1].set_xlabel("Y position (px)")
    axs[0,1].set_ylabel("Score (px)")

    #decision polots
    #TODO: put predicted lines on here?
    axs[1, 0].set_title("Horizontal super module decision")
    axs[1, 0].scatter(gap_x_1, gaps1, label='gaps')
    axs[1, 0].axhline( np.median(gaps1), color = 'r', label='median')
    axs[1, 0].axhline( np.average(gaps1), color = 'b', label='average')
    axs[1, 0].legend(loc="upper right", prop={'size': 6})
    axs[1,0].set_xlabel("X position (px)")
    axs[1,0].set_ylabel("Gap size (px)")

    axs[1, 1].set_title("Vertical super module decision")
    axs[1, 1].scatter(gap_x_2, gaps2, label='gaps' )
    axs[1, 1].axhline( np.median(gaps2), color = 'r', label='median')
    axs[1, 1].axhline( np.average(gaps2), color = 'b', label='average')
    axs[1, 1].legend(loc="upper right", prop={'size': 6})
    axs[1,1].set_xlabel("Y position (px)")
    axs[1,1].set_ylabel("Gap size (px)")

    axs[0, 0].tick_params(reset=True)
    axs[0, 1].tick_params(reset=True)


In [50]:
def find_super_modules(indir, outdir, name,  shift_info=[[], 1]) :
    global smod_track_num
    global prev_img_end_num
    global failed_imgs
    global prev_hor_end_num
    
    img_in = cv2.imread(f'{indir}{name}.png')
    #read image

    
    base_name = name[:4]
    print(base_name)
    #if gray mask :
    #img_in[img_in == 2] = 0

    #if bgr mask :
    img_in = ConvertMasksBGR(img_in)


    #Undistort image
    imge = UndistortImage(img_in)
    #imge = img_in
    
    #choose output image to be recoloured mask, or overlaid on original
    #line_img = cv2.imread(f'/home/dm3315/Documents/SK/PMT_learning/images/Raw/ring_avg/udist/{name[:8]}.png')
    line_img = imge.copy()
    line_img[line_img < 1] = 100  #is using same input

    
    #Crop to central region
    hor_margin = 200 #cuts on x axis
    vert_margin = 100 #cuts on y axis
    imge = imge[hor_margin:-hor_margin, vert_margin:-vert_margin, :]
    
    #DEBUG
   # print(np.unique(imge))
    #cv2.imshow("img", imge)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    #Scan in x and y a line with the angle above and count the number of pixels that appear in the PMT mask
    x_line_score, y_line_score = get_scores(imge, 0)# angle)
    mult = 2 #for speed
    x1 = np.linspace(0, imge.shape[0], int(imge.shape[0]/mult))
    x2 = np.linspace(0, imge.shape[1], int(imge.shape[1]/mult))

    #Get approximate locations of peaks -  centre of pmt lines
    peak_loc_1, guess_thresh_1 = peak_guess(x1, x_line_score)
    peak_loc_2, guess_thresh_2 = peak_guess(x2, y_line_score)
    
    
    ##DEBUG
    #plt.plot(x1, x_line_score)
    #plt.plot(x2, y_line_score)
    #print(peak_loc_1)
    #print(peak_loc_2)
    
    
    #Peak finding methods, three are implemented at the moment
    print(f"Guess: {len(peak_loc_1)} horizontal gaps above {guess_thresh_1}, {len(peak_loc_2)} vertical above {guess_thresh_2}")

    #Do a gaussian fit with an inital guess from above
   # popt1, pcov1 = curve_fit(peaks_func, x1, x_line_score, p0=peak_loc_1)
   # popt2, pcov2 = curve_fit(peaks_func, x2, y_line_score, p0=peak_loc_2)

    #scipy function
    peak_index_x, _ = find_peaks(np.array(x_line_score),  distance=100, prominence=500, width=10) #\
                                              #height=guess_thresh_1, \
                                              #\ 
                                 #np.abs(np.average(np.diff(peak_loc_1)))*0.5, 
                                               #\
                                              
    
    peak_index_y, _ = find_peaks(np.array(y_line_score), distance=100, prominence=500, width = 10)
                                              #height=guess_thresh_2, \
                                               #\  #np.abs(np.average(np.diff(peak_loc_2)))*0.5, 
                                               #\
                                              

  #  print(peak_index_x)
  #  print(peak_index_y)
    
    #find peak centers with different methods
    peaks_x_1 = x1[peak_index_x]
    peaks_y_1 = x2[peak_index_y]
    
    #guess  method
   # peaks_x_2 = peak_loc_1
   # peaks_y_2 = peak_loc_2 
    peaks_x_2 = []
    peaks_y_2 = []
    for i in peak_loc_1 :
        peaks_x_2.append(i[0])

    for i in peak_loc_2 :
        peaks_y_2.append(i[0])
    
    
    #FIT METHOD
    '''
    fit1 = peaks_func(x1, *popt1)
    fit2 = peaks_func(x2, *popt2)    
    peaks_x_3 = []
    peaks_y_3 = []
    for i in range(0, len(popt1), 3) :
        peaks_x_3.append(popt1[i])
    
    for j in range(0, len(popt2), 3) :
        peaks_y_3.append(popt2[j])
    '''    
    
    #Choose a method
    peaks_x = peaks_x_2
    peaks_y = peaks_y_2

    #get super module lines
    super_x, gaps_x = super_lines(peaks_x, "x")
    super_y, gaps_y = super_lines(peaks_y, "y")
    
    ##TODO: Use tracknum to compare if it is close to smod lines found and then set track_num to whatever is the first one 
    
    #Draw lines on image for reference
    line_img_drawn = draw_lines(line_img, peaks_x, peaks_y, hor_margin, vert_margin,  draw_pmt_lines = True)
    img_vert_lines = []     #[[super_y, num]...]
    img_hor_lines = []
    pred_vert_lines = shift_info[0]
    
    #should only be the case for first image, also catches if drone moves too far in which case will likely be wrong
    if len(pred_vert_lines) == 0 :
        for new_line in super_y :
            prev_img_end_num += 1
            img_vert_lines.append([new_line+ vert_margin, prev_img_end_num])
        for new_hor_line in range(len(super_x)) :
            temp_hor_num = prev_hor_end_num + new_hor_line
            img_hor_lines.append([super_x[new_hor_line]+hor_margin, temp_hor_num+1])
    else :
        #print(pred_vert_lines.shape)
        num_prev = len(pred_vert_lines)
        num_current = len(super_y)
        cor_lines = []   #predicted lines corrected for this image
        cor_num = []     #predicted line numbers
        for ucor_line in range(num_prev) :
                cor_lines.append(pred_vert_lines[ucor_line][0] * np.average(np.diff(super_y)))
                cor_num.append(pred_vert_lines[ucor_line][1])

        print(f'max_lines:{np.max(cor_lines)}')
        print(f'max_num:{np.max(cor_num)}')
        print(f'info:{pred_vert_lines}')
        print(f'old lines:{cor_lines}')
        print(f'new_lines {super_y}  {super_x}')


        max_prev_line = np.max(cor_lines)
        max_prev_num = np.max(cor_num)

        #Label smod lines
        #loop through lines in current image, see if a previous line matches, if so give it that number
        #if not increase the global most recent number and assign that number
        #if a line ends up having no number, call image a failure
        #not sure what to do in this case, maybe skip image?
        
        hor_line_num = 1
        for hor_line in super_x :
            #TODO: generalise this
            img_hor_lines.append([hor_line+hor_margin, hor_line_num])
            hor_line_num += 1
            
            
        for line in super_y : #pred_vert_lines = [[line, number]..]
            #TODO: Track numbers from previous image
            #TODO: Only one direction at the moment, will have to do reverse at some point
            
            margin = np.average(np.diff(super_y))*(3/8) # is 1.5 pmts hopefully
            num_assigned = False
            for prev_line in range(len(cor_lines)) :
                if abs(cor_lines[prev_line] - line) < margin :
                    line_num = cor_num[prev_line]
                    img_vert_lines.append([line+vert_margin, cor_num[prev_line]])
                    num_assigned = True
                        
            if line > max_prev_line + margin  :
                prev_img_end_num += 1
                img_vert_lines.append([line, prev_img_end_num])
                num_assigned = True
            
            if not num_assigned :
                failed_imgs.append(f'{base_name}')
                img_vert_lines.append([line, 0])

                #TODO: if image failes, skip image? manually ammend?
                
                
                
        
        #draw_previous numbers and lines passed through
        for num_prev_line in range(len(cor_lines)) :
            cv2.line(line_img_drawn, (int(cor_lines[num_prev_line]), 0), (int(cor_lines[num_prev_line]), line_img.shape[0]),  color=[255, 255, 255] , thickness=2)
            cv2.putText(line_img_drawn, f'{cor_num[num_prev_line]}', (int(cor_lines[num_prev_line]), int(line_img.shape[0]/3)), \
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (255,255,255), 2)
    
    #draw current numbers from returned values (img_vert_lines)
    for num_current_line in img_vert_lines:
        cv2.putText(line_img_drawn, f'{num_current_line[1]}', (int(num_current_line[0]), int(200)), \
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 2)

    for num_current_line in img_hor_lines:
        cv2.putText(line_img_drawn, f'{num_current_line[1]}', (200, int(num_current_line[0])), \
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 2)
        
    #Label PMTs
    #TODO: move array converstion up
    img_hor_lines = np.asarray(img_hor_lines)   #smod horizontal lines
    img_vert_lines = np.asarray(img_vert_lines)   #smod vertical lines
    
    peaks_x_cor = np.array([ucor_x + hor_margin for ucor_x in peaks_x])   #pmt horizontal lines
    peaks_y_cor = np.array([ucor_y + vert_margin for ucor_y in peaks_y])  #pmt vertical lines
    print(f'hor_lines : {img_vert_lines[:,0]}')
    
    
   # Work out the labels for pmts in the images based 
    id_info = np.zeros((len(peaks_x), len(peaks_y), 6)) # for each crossing of pmt lines: [xpos, ypos, smod_x, smod_x, smod_y, smod_pos_x, smod_pos_y]
    for x in range(len(peaks_x)) :
        for y in range(len(peaks_y)) :
            id_info[x, y, 0] = peaks_x_cor[x]
            id_info[x, y, 1] = peaks_y_cor[y]
            
            #Super module ID
            #horizontal number
            if peaks_x_cor[x] < np.min(img_hor_lines[:,0]) :
                id_info[x, y, 3] = img_hor_lines[0,1] - 1
                id_info[x, y, 5] = 3 - np.size(np.where(peaks_x_cor < np.min(img_hor_lines[:,0])))  + x

            else :
                indx = np.max(np.where(img_hor_lines[:,0] < peaks_x_cor[x]))
               # print(f'indx x {indx} {img_hor_lines[indx, 1]} {np.where(img_hor_lines[:,0] < peaks_x_cor[x])}')
                id_info[x, y, 3] = img_hor_lines[indx, 1]
                id_info[x, y, 5] = np.size(np.where((peaks_x_cor > img_hor_lines[indx,0]) & (peaks_x_cor < peaks_x_cor[x])))

            #vertical line numbering
            if peaks_y_cor[y] < np.min(img_vert_lines[:,0]) :
                id_info[x, y, 2] = img_vert_lines[0,1] - 1
                id_info[x, y, 4] = 4 - np.size(np.where(peaks_y_cor < np.min(img_vert_lines[:,0])))  + y

            else : 
                indx = np.max(np.where(img_vert_lines[:,0] < peaks_y_cor[y]))
                #print(f'indx y {indx} : {img_vert_lines[indx, 0]} : {np.where((img_vert_lines[indx,0]< peaks_y_cor))} : {np.where((img_vert_lines[indx,0]< peaks_y_cor) & (peaks_y_cor < peaks_y_cor[y]))}')
                #print(f'where y {np.where((img_vert_lines[indx,0]< peaks_y_cor) & (peaks_y_cor < peaks_y_cor[y]))}')
                id_info[x, y, 2] = img_vert_lines[indx, 1]
                id_info[x, y, 4] = np.size(np.where((img_vert_lines[indx,0]< peaks_y_cor) & (peaks_y_cor < peaks_y_cor[y])))

   # print(id_info)
    
    #draw pmt ids
    for b in id_info :
        for a in b :
            #print(a)
            cv2.putText(line_img_drawn, f'{a[2:]}', (int(a[1])-100, int(a[0])+100), \
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    
    #Write peak locations to file for use in feature finding
    #[name:pmt_locs_x:pmt_locs_y:super_x:super_y]
    pmt_loc = f'{base_name}:{[x+hor_margin for x in peaks_x]}:{[y+vert_margin for y in peaks_y]}:{super_x}:{super_y}'
    pmt_file = open(f'{outdir}pmt_loc.txt', 'a')
    pmt_file.write(f'{pmt_loc} \n')
    pmt_file.close()
    
    
    #compare to validation
    num_cor = 0
    if base_name in val_coords :
        #print([np.array(val_coords[base_name]['vert_lines']) + hor_margin, super_y])
        #print([np.array(val_coords[base_name]['hor_lines']) + vert_margin, super_x])
        for i in super_x :
            for j in val_coords[base_name]['hor_lines'] :
                #print(f'vert:{i} {j}')
                cv2.line(line_img_drawn, (0, j), (line_img.shape[1], j),  color=[255, 0, 0] , thickness=2)
                if abs(i - (j-hor_margin)) < 200 :
                    print([i, (j-hor_margin)])
                    num_cor += 1
        
        for k in super_y :
            for l in val_coords[base_name]['vert_lines'] :
                cv2.line(line_img_drawn, (l, 0), (l, line_img.shape[0]),  color=[255, 0, 0] , thickness=2) #PUT IN ANGLE CORRECTION
                #print(f'hor:{k} {l}')
                if abs(k - (l-vert_margin)) < 200 :
                    print([k, (l-vert_margin)])
                    num_cor += 1                    
        #print(num_cor)
    #plot all PMT lines
    print(f"Found {len(gaps_x)} horizontal gaps")
    print(f"Found {len(gaps_y)} vertical gaps")

    num_modules =  len(gaps_y)
   # print(super_x)

   # plt.show()
    cv2.imwrite(f"{outdir}{name}_lines.png", line_img_drawn)
    
    plot_data(x1, np.array(x_line_score), peaks_x, x2, np.array(y_line_score), peaks_y)
    plt.savefig(f'{outdir}{name}_decisions.png')
    plt.close()
    #cv2.destroyAllWindows()
    #super_y = [[line, num]...]
    return num_modules, num_cor, img_vert_lines, super_x

In [51]:
def get_img_info(img_start, img_end) :
    #Finds image name in drone data and returns the useful bits in order of images taken
    #theres some play in the data which could be accounted for here
    #TODO:validation with epoch
    csv_file = "/home/dm3315/Documents/SK/PMT_learning/images/Raw/SK_TOW_2020_Image_Database-ImageDroneData.csv"
    rows = []
    #img_start = 4147
    #img_end = 4656
    
    with open(f'{csv_file}') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
    #print(readCSV)
        for row in readCSV:
            rows.append(row)
    
    rows = rows[img_start - 1:img_end]

    if len(rows)%10 != 0 :
        print(broken)
        exit()
    img_order = []
    img_info = {}
    
    #Have to manually count index of information and add to dict
    for i in rows :
        img_info[f'{i[2][:-4]}'] = {'survey':i[0],'epoch':i[7],'depth':i[15], 'yaw':i[17], 'pitch':i[18], 'roll':i[19] }
        img_order.append(f'{i[2][:4]}')

    return img_info, np.unique(img_order)

In [52]:
#info = get_img_info(4147, 4656)
#print(info)
#yaw = [] 
#for i in info :
    #print(i['name'])
#    yaw.append(int(i['yaw']))
#print(np.sort(np.unique(yaw)))
#print(len(np.unique(yaw)))


#Class structure to store info
#class Image :
    

In [53]:
def reproject(init_vert_lines, init_hor_lines, init_yaw, end_yaw ) :
    #function projects lines from one image to next, 
    #TODO: do same with depth for moving down tank to next ring
    
    global smod_track_lines
    global smod_track_num
    global ring_start_yaw
    #TODO: line up images by looking at horizontal lines and label them
    #todo: adjust guess by rescaling shift based on change in super module size
    #TODO: account for getting back to start of ring

    
    
    just_vert_lines = []
    for vert_line in range(len(init_vert_lines)) :
        just_vert_lines.append(init_vert_lines[vert_line][0])
    
    vert_smod_px = np.average(np.diff(just_vert_lines))
    hor_smod_px = np.average(np.diff(init_hor_lines))
    
    scale_rat = vert_smod_px/hor_smod_px
    smod_len = 9.6 #degrees
    delta_y = int(end_yaw) - int(init_yaw)
    
    if abs(delta_y) > 340:
        mag_delta_y = abs(360-delta_y)
        sign_delta_y = delta_y/abs(delta_y)
        delta_y = mag_delta_y * sign_delta_y
        
    shift_px = (delta_y / smod_len) #* scale_rat  #this is the fraction of a super module moved to the next image
    
    

    print(f'smod_size:{vert_smod_px}')
    print(f'aspect_ratio:{scale_rat}')
    print(f'shift:{shift_px}')
    print(f'initial lines:{init_vert_lines}')
    
    projected_vert_lines = []
    if shift_px < 0 :
        for i in range(len(init_vert_lines)) :
            projected_vert_lines.append([(init_vert_lines[i][0]/vert_smod_px) + shift_px, init_vert_lines[i][1]] )
    else :
        for i in range(len(init_vert_lines)) :
            projected_vert_lines.append([(init_vert_lines[i][0]/vert_smod_px) - shift_px, init_vert_lines[i][1]] )
            
            #smod_track_lines.append([init_vert_lines[i], smod_track_num] )

           #         for i in range(len(init_vert_lines)) :
           # projected_vert_lines.append(4000 - (init_vert_lines[i]+shift_px) )
           # smod_track_lines.append([init_vert_lines[i], smod_track_num] )
            
    #return fraction of pmt moved
    return [projected_vert_lines, hor_smod_px]

In [ ]:
#Need segemnted files for input (could be done with just points maybe?
#need the csv file with information
#Raw images useful for overlay
#outputs txt file with positions of pmts and super modules horizontally and vertically


import time 
start_time = time.time()

method  ='guess_meth'
seg_files = 'ring_avg'

work_dir = f"/home/dm3315/Documents/SK/PMT_learning/images/datasets/substacks_far/output/{seg_files}/"
#out_dir = f"/home/dm3315/Documents/SK/PMT_learning/images/ring_reco/four_gaps/{method}/"
out_dir = f"/home/dm3315/Documents/SK/PMT_learning/images/datasets/substacks_far/output/{seg_files}/smodule/{method}/"

#indir = "/home/dm3315/Documents/SK/PMT_learning/images/Raw/ring_avg/"
f = []
for (dirpath, dirnames, filenames) in os.walk(f'{work_dir}'):
    f.extend(filenames)
    break
f = [x[:-4] for x in f]

#info, order = get_img_info(4577, 4656)
info, order = get_img_info(4147, 4656)
#print(info)
tot_mods = 0
tot_cor = 0
img_num = 1

#global variables that may not be used
smod_track_num = 0
smod_count_num = 0
prev_img_end_num = 0
prev_hor_end_num = 0
smod_count_lines = [] #just counting lines
smod_track_lines = [] #applying unique numbers
failed_imgs = []

#get list of images in order they were taken (i.e. going round the ring)
os.chdir(f"{work_dir}")
#print(info)
pmt_info_file = open(f'/home/dm3315/Documents/SK/PMT_learning/images/datasets/substacks_far/output/{seg_files}/smodule/{method}/pmt_loc.txt', 'w')
pmt_info_file.close()
num_images = len(order) - 1
print(f'num images:{num_images}')

#loop em
i = 0
while i < num_images :
    
    #Get list of image names from files 
    if len(glob.glob(f"{order[i]}*")) > 1 :
        print(f'Duplicate images: {glob.glob(f"{order[i]}*")}')
        continue
        
    print(order[i])
    print(order[i+1])
    filename = glob.glob(f"{order[i]}*")[0]
    
    if len(glob.glob(f"{order[i+1]}*")) == 0 :
        order = np.delete(order, i+1)
        num_images -= 1
    next_filename = glob.glob(f"{order[i+1]}*")[0]
    print(f'\n Processing image {img_num} of {len(f)}: {filename[:-4]}')

    
    #TODO:change this to do multiple rings
    if img_num == 1 :
        smod_shift = [[], 1]
        ring_start_yaw = None
    num, num_cor, vert, hor = find_super_modules(work_dir, out_dir, filename[:-4], smod_shift)
    tot_mods += num
    tot_cor += num_cor
    print(f'num_cor = {tot_cor}')
    
    #TODO:decide if ring is finished:
    
    
        #get start of ring and catch end of ring
    if seg_files=='ring_avg':
        yaw_1 = int(info[filename[:8]]['yaw'])
        yaw_2 = int(info[next_filename[:8]]['yaw'])
        depth_1 = info[filename[:8]]['depth']
        depth_2 = info[next_filename[:8]]['depth']
        if yaw_2 - yaw_1 < 0 :
            if ring_start_yaw == None :
                ring_start_yaw = yaw_1
            else :
                if (abs(yaw_1 - ring_start_yaw < 10)) and (yaw_1 < ring_start_yaw) and (img_num > 20) :
                    print(f'ring complete at depth :{depth_1}')
                    #ring_start_yaw = None
                    #TODO: get to next ring depth, project initial img lines back to last img lines and label
                    #TODO: reproject horizontal line labels to next depth
                    #TODO: other way around ring?
                    
        
        #Reproject image horizontally to next one
        print(f'yaws: {yaw_1}, {yaw_2}')
        print(f'names: {order[i]}, {order[i+1]}')
        #scaled by horizontal spacing, basically aspect ratio
        smod_shift = reproject(vert, hor, yaw_1, yaw_2 )
        print(f'smods moved:{smod_shift}')
    i += 1
    img_num += 1
    
pmt_info_file.close()
print(f"found {prev_img_end_num} Super modules horizontally in ring. {tot_cor} of 64 are correct")
print(f'Images with failed labels: {failed_imgs}')

print(f'time taken = {time.time() - start_time} seconds')

num images:51
B755
B756

 Processing image 1 of 51: B7550546_pred_substacks_far_unet.19
B755
Guess: 7 horizontal gaps above 1534.0459574468084, 9 vertical above 949.7852631578947
hor_lines : [1709.84728805 3133.59662981]
[1048.3922487223167, 1073]
[1609.8472880463405, 1603]
[3033.596629805161, 3010]
Found 1 horizontal gaps
Found 1 vertical gaps
num_cor = 3
yaws: 165, 159
names: B755, B756
smod_size:1423.7493417588203
aspect_ratio:1.3698003275795534
shift:-0.625
initial lines:[[1.70984729e+03 1.00000000e+00]
 [3.13359663e+03 2.00000000e+00]]
smods moved:[[[0.5759468506121246, 1.0], [1.5759468506121244, 2.0]], 1039.3845826235092]
B756
B757

 Processing image 2 of 51: B7560556_pred_substacks_far_unet.19
B756
Guess: 7 horizontal gaps above 1523.3634042553192, 9 vertical above 943.6626315789474
max_lines:2205.9106496114214
max_num:2.0
info:[[0.5759468506121246, 1.0], [1.5759468506121244, 2.0]]
old lines:[806.1739460832484, 2205.9106496114214]
new_lines [468.24644549763036, 1867.983149025803

Guess: 7 horizontal gaps above 1350.3804255319149, 10 vertical above 835.69
max_lines:2196.581999194192
max_num:9.0
info:[[0.5704047949965254, 0.0], [1.5704047949965254, 9.0]]
old lines:[797.8458222589632, 2196.581999194192]
new_lines [945.9978936282255, 2344.7340705634547]  [1039.8850085178876, 2082.7725724020443]
hor_lines : [1045.99789363 2444.73407056]
[1039.8850085178876, 1059]
[2344.7340705634547, 2210]
Found 1 horizontal gaps
Found 1 vertical gaps
num_cor = 18
yaws: 111, 106
names: B765, B766
smod_size:1398.7361769352292
aspect_ratio:1.3412147439228645
shift:-0.5208333333333334
initial lines:[[1045.99789363    0.        ]
 [2444.73407056    9.        ]]
smods moved:[[[0.2269830961523981, 0.0], [1.2269830961523982, 9.0]], 1042.8875638841566]
B766
B767

 Processing image 12 of 51: B7660656_pred_substacks_far_unet.19
B766
Guess: 7 horizontal gaps above 1139.2553191489362, 9 vertical above 704.8
max_lines:1556.0200322848934
max_num:9.0
info:[[0.2269830961523981, 0.0], [1.22698309615

In [ ]:
#Testing below here

In [20]:
inpath = '/home/dm3315/Documents/SK/PMT_learning/images/Raw/large_labeled/frames/B5310.png'   
imge = cv2.imread(f'{inpath}')
out = UndistortImage(imge)

cv2.imwrite("/home/dm3315/Documents/SK/PMT_learning/images/Raw/undistorted/orig.png", imge)
cv2.imwrite("/home/dm3315/Documents/SK/PMT_learning/images/Raw/undistorted/undist.png", out)
#cv2.waitKey(0) 

True

In [ ]:
import os
#indir = "/home/dm3315/Documents/SK/PMT_learning/images/Raw/ring_avg/"
f = []
for (dirpath, dirnames, filenames) in os.walk(f'{indir}'):
    f.extend(filenames)
    break

for k in f :
    orig = cv2.imread(f"{indir}{k}")
    undist = UndistortImage(orig)
    cv2.imwrite(f"{indir}udist/{k}", undist)

In [ ]:
def ConvertMasksBGR(BGR_img) :
    lower_pmts = np.array([254, 0, 0], dtype = "uint16")
    upper_pmts = np.array([255,0,0], dtype = "uint16")
    pmt_mask = cv2.inRange(BGR_img, lower_pmts, upper_pmts)
    pmt_mask[pmt_mask == 255] = 1
    return pmt_mask

inpath = "/home/dm3315/Documents/SK/PMT_learning/images/datasets/bolt_pmt_blur/output/ring_avg/B7550546.png_pred_noverlay.png"
img = cv2.imread(f"{inpath}")
print(np.unique(img))

pmts = ConvertMasksBGR(img)
#pmts_gray = cv2.cvtColor(pmts, cv2.COLOR_BGR2GRAY)
print(np.unique(pmts))
cv2.imshow("img", pmts)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#apply convolution 

#prepare the 5x5 shaped filter
kernel = np.array([[1, 1, 1, 1, 1], 
                   [1, 1, 1, 1, 1], 
                   [1, 1, 1, 1, 1], 
                   [1, 1, 1, 1, 1], 
                   [1, 1, 1, 1, 1]])
kernel = np.round(kernel/sum(kernel))

#filter the source image
img_rst = cv2.filter2D(img_in,-1,kernel)

#save result image
cv2.imwrite('/home/dm3315/Documents/SK/PMT_learning/images/conv.png',img_rst)
